# ĐỒ ÁN CUỐI KỲ (NMKHDL - CQ2018/2)
### Giảng viên hướng dẫn: Thầy Trần Trung Kiên
### Trợ giảng: Thầy Hồ Xuân Trường

__Thông tin nhóm__: 

* STT: 46

* Họ và tên sinh viên: Nguyễn Hữu Huân - MSSV: 1712466

* Họ và tên sinh viên: Đặng Hữu Thắng - MSSV: 18120555

---

## Import

In [62]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# for preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

# for model picking
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

----

## Thu thập dữ liệu

Dữ liệu mà nhóm em dùng cho đồ án cuối kỳ được tự thu thập từ trang [Booking.com](https://booking.com) - Trang web về du lịch cung cấp dịch vụ đặt chỗ, đặt phòng khách sạn ở trực tuyến trên toàn thế giới. Dữ liệu nhóm em thu thập chứa thông tin về chỗ ở (bao gồm cả khách sạn, nhà nghỉ, homestay,..) ở 4 địa điểm du lịch của Việt Nam: Hà Nội, Đà Nẵng, Đà Lạt, Thành phố Hồ Chí Minh.

Dữ liệu trên trang web này được phép thu thập, được mô tả cụ thể thông qua file robots.txt.

Phần source code cho quá trình cào dữ liệu về  được lưu ở file notebook [crawling_data.ipynb](https://github.com/ref-to-uploaded-notebook).

Toàn bộ dữ liệu thô sau khi cào về lưu ở file: "full_data.csv".

---

## Khám phá dữ liệu (đủ để xác định câu hỏi) + tiền xử lý 

Do file "full_data.csv" là tất cả dữ liệu mà nhóm em tự cào được, nên nhóm sẽ cần khám phá một ít trên dữ liệu để đưa ra câu hỏi cần trả lời, sau đó tiến hành kiểm tra, làm sạch một vài vấn đề cơ bản (vì tất cả hoàn toàn là dữ liệu thô) và ngay lập tức tách tập validation và test ra khỏi dữ liệu. Như đã được thầy đề cập ở BT03, điều này để tránh hiểu quá sâu dữ liệu, làm mất đi tính khách quan khi đánh giá kết quả.

In [3]:
data_df =pd.read_csv('../datasets/full_data.csv')
data_df.sample(n=10)

,City,Diện tích,Đồ vệ sinh cá nhân miễn phí,Vòi sen,Áo choàng tắm,Két an toàn,Nhà vệ sinh,Khăn tắm,Bàn làm việc,Khu vực tiếp khách,TV,Dép,Tủ lạnh,Điện thoại,Hệ thống sưởi,Máy sấy tóc,Sàn trải thảm,Ấm đun nước điện,Truyền hình cáp,Dịch vụ báo thức,Tủ hoặc phòng để quần áo,Giá treo quần áo,Nhìn ra thành phố,Bồn tắm,Điều hòa không khí,Phòng tắm riêng,TV màn hình phẳng,Hệ thống cách âm,Minibar,WiFi miễn phí,Ghế sofa,Máy fax,Phòng thay quần áo,Nhìn ra hồ bơi,Bếp,Máy giặt,Bồn tắm hoặc Vòi sen,Đồng hồ báo thức,Bàn ăn,Bồn tắm spa,Ổ điện gần giường,Toilet phụ,Dịch vụ báo thức,Giấy vệ sinh,Phòng tắm riêng trong phòng,Tiện nghi ủi,Giường cực dài (> 2 mét),Bếp nhỏ,Khu vực phòng ăn,Tầm nhìn ra khung cảnh,Truyền hình vệ tinh,Bàn ủi,Ban công,Nhìn ra sông,Quyền sử dụng Executive Lounge,Sàn lát gỗ,Ra trải giường,Không gây dị ứng,Máy pha trà/cà phê,Khăn tắm/Bộ khăn trải giường (có thu phí),Máy sấy quần áo,Nhìn ra địa danh nổi tiếng,Phòng tắm phụ,Có phòng thông nhau qua cửa nối,Chậu rửa vệ sinh (bidet),Lối vào riêng,Các tầng trên đi lên bằng thang máy,Xe lăn có thể đi đến mọi nơi trong toàn bộ khuôn viên,Nhìn ra vườn,Máy vi tính,Giường sofa,Giá phơi quần áo,Két an toàn cỡ laptop,Giường xếp,Quạt máy,Chăn điện,Sản phẩm lau rửa,Ghế cao dành cho trẻ em,Cửa an toàn cho trẻ nhỏ,Máy pha Cà phê,Sàn lát gạch/đá cẩm thạch,Bàn ủi li quần,Hoàn toàn nằm ở tầng trệt,Các tầng trên chỉ lên được bằng cầu thang,Nắp che ổ cắm điện an toàn,Đồ bếp,Đầu đĩa CD,Máy điều hòa độc lập cho từng phòng,Nước rửa tay,Đầu đĩa DVD,Hồ bơi trên sân thượng,Ổ cắm cho iPod,Hồ bơi có tầm nhìn,Trò chơi board game/giải đố,Sách,đĩa DVD và nhạc cho trẻ em,Có lắp đặt máy lọc không khí,Thiết bị báo carbon monoxide,Lò vi sóng,Máy nướng bánh mỳ,Lò sưởi,Lò nướng,Bếp nấu,Đài radio,Khu vực ăn uống ngoài trời,Phòng xông hơi,Toilet chung,Bàn ghế ngoài trời,Bể sục,Hồ bơi riêng,Sân trong,Hướng nhìn sân trong,price
9290,Da Nang,75 m²,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,0,0,1,1,1,1,1,1,1,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1.811.066
8366,Da Nang,110 m²,1,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,0,0,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,1,1,1,1,1,0,0,1,0,1,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,3.638.810
1465,TP.Ho Chi Minh,25 m²,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,338.978
8782,Da Nang,32 m²,1,0,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,1,1,1,0,0,1,0,1,1,1,1,0,0,0,0,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.655.280
6371,Ha Noi,20 m²,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,664.200
8251,Da Nang,65 m²,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,0,0,1,1,1,0,1,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1.217.700
13944,Da Lat,18 m²,1,1,0,0,1,1,0,0,1,1,1,1,0,1,0,1,1,0,0,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,202.500
12595,Da Lat,28 m²,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,987.840
12829,Da Lat,18 m²,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,

### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [4]:
data_df.shape

(14316, 113)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Quan sát sơ qua thì ta thấy mỗi dòng chứa thông tin của một chỗ ở. Mà cụ thể hơn, do dữ liệu được nhóm em cào nên đó là chỗ ở cho thuê cho 2 người, trong 1 ngày 1 đêm, được cào trong giai đoạn từ ngày ... đến ngày.

Nhìn sơ qua (bằng cách chạy `data_dfsample()` nhiều lần) thì thấy có vẻ có một số dòng bị vấn đề ở cột "Diện tích". Cụ thể, thay vì chứa thông tin về "x m²" thì sẽ là "Phòng tắm riêng trong phòng", "Ban công", "Điều hòa không khí",... 

Có thể đây là lỗi gặp phải trong quá trình nhóm em cào tự động, những dòng này cần phải được loại bỏ. Sau đó, sẵn tiện thì lược bỏ luôn "m2" trong cột này, chuyển sang dạng số luôn.

In [5]:
data_df.drop(data_df[data_df['Diện tích'].str.find('m²') == -1].index,inplace=True)
data_df.shape

(13397, 113)

In [6]:
data_df['Diện tích'] = data_df['Diện tích'].str[:-3]
data_df['Diện tích'] = pd.to_numeric(data_df['Diện tích'], errors='coerce')
data_df['Diện tích'].dtypes

dtype('int64')

### Dữ liệu có các dòng bị lặp không?

In [7]:
data_df.index.duplicated().sum()

0

### Mỗi cột có ý nghĩa gì?

Vì hầu hết thông tin trên booking.com đều đã được dịch sang Tiếng Việt nên các cột nhìn vào có thể hiểu được nó chứa thông tin về gì. Ngoài thông tin về "price", "City", "Diện tích" thì các thông tin còn lại của booking đều hiển thị dưới dạng: 1 - có, 0 - không. Nên dữ liệu ở các cột còn lại đều là thuộc loại binary.

In [8]:
list(data_df.columns)

['City',
 'Diện tích',
 'Đồ vệ sinh cá nhân miễn phí',
 'Vòi sen',
 'Áo choàng tắm',
 'Két an toàn',
 'Nhà vệ sinh',
 'Khăn tắm',
 'Bàn làm việc',
 'Khu vực tiếp khách',
 'TV',
 'Dép',
 'Tủ lạnh',
 'Điện thoại',
 'Hệ thống sưởi',
 'Máy sấy tóc',
 'Sàn trải thảm',
 'Ấm đun nước điện',
 'Truyền hình cáp',
 'Dịch vụ báo thức',
 'Tủ hoặc phòng để quần áo',
 'Giá treo quần áo',
 'Nhìn ra thành phố',
 'Bồn tắm',
 'Điều hòa không khí',
 'Phòng tắm riêng',
 'TV màn hình phẳng',
 'Hệ thống cách âm',
 'Minibar',
 'WiFi miễn phí',
 'Ghế sofa',
 'Máy fax',
 'Phòng thay quần áo',
 'Nhìn ra hồ bơi',
 'Bếp',
 'Máy giặt',
 'Bồn tắm hoặc Vòi sen',
 'Đồng hồ báo thức',
 'Bàn ăn',
 'Bồn tắm spa',
 'Ổ điện gần giường',
 'Toilet phụ',
 'Dịch vụ báo thức',
 'Giấy vệ sinh',
 'Phòng tắm riêng trong phòng',
 'Tiện nghi ủi',
 'Giường cực dài (> 2 mét)',
 'Bếp nhỏ',
 'Khu vực phòng ăn',
 'Tầm nhìn ra khung cảnh',
 'Truyền hình vệ tinh',
 'Bàn ủi',
 'Ban công'

In [9]:
for col in set(data_df.columns) - set(["price", "City", "Diện tích"]):
    print(data_df[col].unique())

[1 0]
[0 1]
[0 1]
[1 0]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[1 0]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[1 0]
[0 1]
[1 0]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]


---

## Đưa ra câu hỏi cần trả lời

Với dữ liệu này, nhóm em đã xác định từ đầu cột output sẽ là "price" - Bài toán regression. Do đó, câu hỏi cần được trả lời sẽ là:

*Output - giá thuê chỗ ở (cho 2 người, trong 1 ngày 1 đêm, đơn vị: VNĐ) -* được tính từ *input - các thông tin của chỗ ở* theo công thức nào?

Việc tìm ra câu trả lời cho câu hỏi này góp phần nào tham khảo vào quá trình đưa ra quyết định cho cả 2 phía: bên cho thuê và bên thuê chỗ ở khi đi du lịch. Bên cho thuê có thể dựa vào đó để đưa ra giá tiền thuê sao cho phù hợp với thị trường, dựa trên những tiêu chí/ thông tin của tài sản. Cũng như cần chuẩn bị những tiêu chí gì để "tối ưu" giá cho thuê lên. Ngược lại, bên đi thuê sẽ dựa theo các tiêu chí mình cần để chuẩn bị ngân sách cho chỗ ở khi du lịch... 

Ở giai đoạn này, thông tin của chỗ ở - input sẽ cần được lọc bớt. Vì hiện tại dữ liệu có khá nhiều cột và nhóm cảm thấy một số cột là không cần thiết/ không ảnh hưởng tới output (ít nhất là với góc độ của thị trường Việt Nam).  

In [10]:
drop_cols = ['Bể sục','Máy fax', 'Ổ cắm cho iPod', 'Trò chơi board game/giải đố','Không gây dị ứng','Dịch vụ báo thức', 'Lối vào riêng', 'Chăn điện', 'Sản phẩm lau rửa',  'Đầu đĩa CD',  'Đầu đĩa DVD', ' đĩa DVD và nhạc cho trẻ em', 'Thiết bị báo carbon monoxide', 'Đài radio', 'Dịch vụ báo thức']
data_df.drop(drop_cols, axis=1, inplace=True)
data_df.shape

(13397, 98)

---

## Khám phá dữ liệu (để biết tách các tập) + tiền xử lý

Ở bước này, dữ liệu ở cột output cần được khám phá một ít để phục vụ cho việc tách tập:


* Cột output có kiểu dữ liệu gì? 

In [11]:
# Cột output có kiểu dữ liệu gì?
data_df['price'].dtype

dtype('O')

Là kiểu object - string. Cột này cần được tiền xử lý, chuyển sang dạng số vì là bài toán regression. Ngoài ra cần bỏ luôn "m²" và những dấu "," trong đó.

In [12]:
data_df['price'] = data_df['price'].str.replace('.', '')
data_df['price'] = pd.to_numeric(data_df['price'],errors='coerce')
data_df['price'].dtype

dtype('int64')

* Cột này có giá trị thiếu không?

In [13]:
data_df['price'].isna().sum()

0

---

## Tiền xử lý (tách các tập)

Sau khi đã khám phá và tiền xử lý 1 ít với dữ liệu, thì bây giờ là lúc tách tập test và validation ra khỏi dữ liệu.

In [14]:
# Tách X và y
y_sr = data_df["price"] # sr là viết tắt của series
X_df =data_df.drop("price", axis=1)

In [15]:
# Tách tập huấn luyện và tập test theo tỉ lệ 80%:20%
train_X_df, test_X_df, train_y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.2, 
                                                                  random_state=0)
# Tiếp tục từ tập huấn luyện tách ra tập huấn luyện và tập validation 80%:20%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.2, 
                                                                  random_state=0)
                                                            

In [16]:
train_X_df.shape, train_y_sr.shape

((10717, 97), (10717,))

In [17]:
 val_X_df.shape, val_y_sr.shape

((2680, 97), (2680,))

In [18]:
test_X_df.shape, test_y_sr.shape

((2680, 97), (2680,))

---

## Khám phá dữ liệu (tập huấn luyện)

### Mỗi cột input hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [19]:
train_X_df.dtypes

City                                                     object
Diện tích                                                 int64
Đồ vệ sinh cá nhân miễn phí                               int64
Vòi sen                                                  int64
Áo choàng tắm                                          int64
Két an toàn                                             int64
Nhà vệ sinh                                               int64
Khăn tắm                                                  int64
Bàn làm việc                                           int64
Khu vực tiếp khách                                     int64
TV                                                        int64
Dép                                                      int64
Tủ lạnh                                                 int64
Điện thoại                                              int64
Hệ thống sưởi                                          int64
Máy sấy tóc                          

Các cột đều có kiểu dữ liệu phù hợp.

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

Chỉ có cột "Diện tích" là kiểu dạng số . 

In [20]:
num_cols = ['Diện tích']
X_df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
X_df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Diện tích
missing_ratio,0.0
min,15.0
lower_quartile,24.0
median,30.0
upper_quartile,42.0
max,1200.0


### Với mỗi cột input có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

In [21]:
cat_cols = list(set(train_X_df.columns) - set(num_cols))
X_df =train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
X_df.agg([missing_ratio, num_values, value_ratios])

,Phòng tắm riêng,Nhìn ra hồ bơi,Các tầng trên chỉ lên được bằng cầu thang,City,Hệ thống sưởi,Bếp nhỏ,Khăn tắm,Máy pha trà/cà phê,Ban công,Bàn ăn,Két an toàn cỡ laptop,Lò sưởi,Ấm đun nước điện,Truyền hình vệ tinh,Bàn làm việc,Đồ vệ sinh cá nhân miễn phí,Có lắp đặt máy lọc không khí,Đồng hồ báo thức,Chậu rửa vệ sinh (bidet),TV màn hình phẳng,Nhìn ra thành phố,Cửa an toàn cho trẻ nhỏ,Khu vực phòng ăn,Giường cực dài (> 2 mét),Bếp nấu,Điện thoại,Nhìn ra vườn,Sàn lát gỗ,Nhìn ra địa danh nổi tiếng,Minibar,TV,Giường sofa,Giá phơi quần áo,Máy sấy quần áo,Quạt máy,Khu vực tiếp khách,Quyền sử dụng Executive Lounge,Bếp,Hồ bơi riêng,Giấy vệ sinh,Hoàn toàn nằm ở tầng trệt,Ghế cao dành cho trẻ em,Toilet phụ,Phòng xông hơi,Nhìn ra sông,Bồn tắm hoặc Vòi sen,Khăn tắm/Bộ khăn trải giường (có thu phí),Máy nướng bánh mỳ,Giá treo quần áo,Ra trải giường,Nắp che ổ cắm điện an toàn,Sàn trải thảm,Phòng tắm phụ,Toilet chung,Sách,Phòng tắm riêng trong phòng,Lò vi sóng,Ổ điện gần giường,Tủ lạnh,Dép,Sân trong,Máy vi tính,Bồn tắm,Máy giặt,Bồn tắm spa,Hệ thống cách âm,Két an toàn,Có phòng thông nhau qua cửa nối,Bàn ủi,Phòng thay quần áo,Nhà vệ sinh,Ghế sofa,Hồ bơi trên sân thượng,Truyền hình cáp,Xe lăn có thể đi đến mọi nơi trong toàn bộ khuôn viên,Máy pha Cà phê,Bàn ủi li quần,Hồ bơi có tầm nhìn,Tiện nghi ủi,WiFi miễn phí,Giường xếp,Tủ hoặc phòng để quần áo,Tầm nhìn ra khung cảnh,Máy sấy tóc,Hướng nhìn sân trong,Điều hòa không khí,Đồ bếp,Khu vực ăn uống ngoài trời,Các tầng trên đi lên bằng thang máy,Vòi sen,Bàn ghế ngoài trời,Nước rửa tay,Áo choàng tắm,Máy điều hòa độc lập cho từng phòng,Sàn lát gạch/đá cẩm thạch,Lò nướng
missing_ratio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
num_values,2,2,2,4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
value_ratios,"{0: 53.2, 1: 46.8}","{0: 95.3, 1: 4.7}","{0: 77.9, 1: 22.1}","{'Ha Noi': 28.4, 'Da Nang': 24.6, 'TP.Ho Chi M...","{0: 82.3, 1: 17.7}","{0: 79.8, 1: 20.2}","{1: 90.1, 0: 9.9}","{0: 80.1, 1: 19.9}","{0: 65.7, 1: 34.3}","{0: 65.8, 1: 34.2}","{0: 85.6, 1: 14.4}","{0: 90.2, 1: 9.8}","{1: 82.3, 0: 17.7}","{0: 59.0, 1: 41.0}","{1: 79.7, 0: 20.3}","{1: 88.1, 0: 11.9}","{0: 94.3, 1: 5.7}","{0: 74.6, 1: 25.4}","{0: 53.5, 1: 46.5}","{1: 88.7, 0: 11.3}","{1: 53.9, 0: 46.1}","{0: 81.0, 1: 19.0}","{0: 72.0, 1: 28.0}","{0: 83.4, 1: 16.6}","{0: 95.7, 1: 4.3}","{1: 65.1, 0: 34.9}","{0: 82.6, 1: 17.4}","{0: 55.4, 1: 44.6}","{0: 87.8, 1: 12.2}","{1: 63.0, 0: 37.0}","{1: 82.7, 0: 17.3}","{0: 84.9, 1: 15.1}","{0: 72.1, 1: 27.9}","{0: 79.7, 1: 20.3}","{0: 63.6, 1: 36.4}","{1: 56.2, 0: 43.8}","{0: 89.9, 1: 10.1}","{0: 78.3, 1: 21.7}","{0: 86.6, 1: 13.4}","{1: 92.5, 0: 7.5}","{0: 91.9, 1: 8.1}","{0: 92.7, 1: 7.3}","{0: 80.3, 1: 19.7}","{0: 87.4, 1: 12.6}","{0: 93.7, 1: 6.3}","{1: 68.2, 0: 31.8}","{0: 68.8, 1: 31.2}","{0: 87.2, 1: 12.8}","{1: 75.7, 0: 24.3}","{1: 71.3, 0: 28.7}","{0: 77.4, 1: 22.6}","{0: 76.8, 1: 23.2}","{0: 91.7, 1: 8.3}","{0: 82.9, 1: 17.1}","{0: 96.9, 1: 3.1}","{0: 58.4, 1: 41.6}","{0: 94.3, 1: 5.7}","{1: 72.5, 0: 27.5}","{1: 60.2, 0: 39.8}","{1: 90.0, 0: 10.0}","{0: 90.9, 1: 9.1}","{0: 98.3, 1: 1.7}","{0: 94.9, 1: 5.1}","{0: 83.4, 1: 16.6}","{0: 96.8, 1: 3.2}","{0: 50.5, 1: 49.5}","{1: 51.0, 0: 49.0}","{0: 90.5, 1: 9.5}","{0: 61.4, 1: 38.6}","{0: 74.9, 1: 25.1}","{1: 93.0, 0: 7.0}","{0: 63.8, 1: 36.2}","{0: 91.8, 1: 8.2}","{1: 68.0, 0: 32.0}","{0: 69.8, 1: 30.2}","{0: 93.4, 1: 6.6}","{0: 89.2, 1: 10.8}","{0: 88.0, 1: 12.0}","{0: 62.9, 1: 37.1}","{1: 79.2, 0: 20.8}","{0: 92.4, 1: 7.6}","{1: 82.5, 0: 17.5}","{0: 89.6, 1: 10.4}","{1: 88.8, 0: 11.2}","{0: 79.3, 1: 20.7}","{1: 77.8, 0: 22.2}","{0: 78.6, 1: 21.4}","{0: 97.5, 1: 2.5}","{1: 53.9, 0: 46.1}","{0: 74.3, 1: 25.7}",

Một số trường có vẻ bị chênh lệch quá nhiều giưa 2 giá trị 0-1. Có thể kể tới như "Giường xếp", "Giấy vệ sinh",... Liệu có cần bỏ các cột này?

---

## Tiền xử lý (tập huấn luyện)

In [22]:
train_X_df.shape

(10717, 97)

In [26]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, may_pha_tra_cf=True,
                bep_an=True, phong_tam=True, ket_sat=True,toilet=True,
                TV=True, dieu_hoa=True,lot_san=True, tu_quan_ao=True,
                view=True, ban_an=True, ui_quan_ao=True, giuong=True,
                tien_ich_cho_tre_em=True, ho_boi=True,
                khu_ngoai_troi=True, tien_ich_phong=True):
        
        self.may_pha_tra_cf = may_pha_tra_cf
        self.bep_an = bep_an
        self.phong_tam = phong_tam
        self.ket_sat = ket_sat
        self.toilet = toilet
        self.TV = TV
        self.dieu_hoa = dieu_hoa
        self.lot_san = lot_san
        self.tu_quan_ao = tu_quan_ao
        self.view = view
        self.ban_an = ban_an
        self.ui_quan_ao = ui_quan_ao
        self.giuong = giuong
        self.tien_ich_cho_tre_em = tien_ich_cho_tre_em
        self.ho_boi = ho_boi
        self.khu_ngoai_troi = khu_ngoai_troi
        self.tien_ich_phong = tien_ich_phong
        
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        df = X_df.copy()
        if self.may_pha_tra_cf:
            col1 = 'Máy pha trà/cà phê'
            col2 = 'Máy pha Cà phê'
            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop(col2, axis=1, inplace=True)
            df = df.rename(columns={col1 : 'may_pha_tra_cf'})
        if self.bep_an:
            col1 = 'Bếp'
            col2 = 'Bếp nhỏ'
            col3 = 'Đồ bếp'
            col4 = 'Bếp nấu'
            col5 = 'Lò vi sóng'
            col6 = 'Máy nướng bánh mỳ'
            col7 = 'Lò nướng'
            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0 and df[col4][i] == 0 and df[col5][i] == 0 and df[col6][i] == 0 and df[col7][i] == 0:
                    pass
                else:
                    if df[col5][i] == 1:
                        if df[col6][i] == 1:
                            if df[col7][i] == 1:
                                df[col1][i] = 3
                            else:
                                df[col1][i] = 2
                        else:
                            df[col1][i] = 1
                    else:
                        df[col1][i] = 1


            df.drop([col2, col3, col4, col5, col6, col7], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'bep_an'})

        if self.phong_tam:
            col1 = 'Vòi sen'
            col2 = 'Bồn tắm'
            col3 = 'Bồn tắm hoặc Vòi sen'
            col4 = 'Bồn tắm spa'
            col5 = 'Phòng tắm riêng trong phòng'
            col6 = 'Phòng tắm phụ'
            col7 = 'Phòng tắm riêng'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0 and df[col4][i] == 0 and df[col5][i] == 0 and df[col6][i] == 0 and df[col7][i] == 0:
                    df[col1][i] = 1
                else:
                    if df[col2][i] == 1 or df[col3][i] == 1 or df[col5][i] == 1 or df[col6][i] == 1 or df[col7][i] == 1:
                        df[col1][i] = 2
                    elif df[col4][i] == 1:
                        df[col1][i] = 3
                    else:
                        df[col1][i] = 1

            df.drop([col2, col3, col4, col5, col6, col7], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'phong_tam'})
            
        if self.ket_sat:
            col1 = 'Két an toàn'
            col2 = 'Két an toàn cỡ laptop'
            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0:
                    pass
                else:
                    if df[col2][i] == 1:
                        df[col1][i] = 2

            df.drop(col2, axis=1, inplace=True)
            df = df.rename(columns={col1 : 'ket_sat'})
        if self.toilet:
            col1 = 'Nhà vệ sinh'
            col2 = 'Toilet phụ'
            col3 = 'Toilet chung'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop([col2, col3], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'toilet'})
        if self.TV:
            col1 = 'TV'
            col2 = 'TV màn hình phẳng'
            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop(col2, axis=1, inplace=True)
            df = df.rename(columns={col1 : 'TV'})
        if self.dieu_hoa:
            col1 = 'Quạt máy'
            col2 = 'Điều hòa không khí'
            col3 = 'Hệ thống sưởi'
            col4 = 'Máy điều hòa độc lập cho từng phòng'
            col5 = 'Có lắp đặt máy lọc không khí'
            col6 = 'Lò sưởi'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0 and df[col4][i] == 0 and df[col5][i] == 0 and df[col6][i] == 0:
                    pass
                else:
                    if df[col2][i] == 1 or df[col4][i] == 1:
                        df[col1][i] = 2
                    if df[col3][i] == 1 or df[col6][i] == 1:
                        df[col1][i] = 3
                    if df[col5][i] == 1:
                        df[col1][i] = 4


            df.drop([col2, col3, col4, col5, col6], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'dieu_hoa'})
        if self.lot_san:
            col1 = 'Sàn trải thảm'
            col2 = 'Sàn lát gạch/đá cẩm thạch'
            col3 = 'Sàn lát gỗ'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0:
                    pass
                else:
                    if df[col2][i] == 1:
                        df[col1][i] = 2
                    if df[col3][i] == 1:
                        df[col1][i] = 3

            df.drop([col2, col3], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'lot_san'})
        if self.tu_quan_ao:
            col1 = 'Tủ hoặc phòng để quần áo'
            col2 = 'Giá treo quần áo'
            col3 = 'Phòng thay quần áo'
            col4 = 'Giá phơi quần áo'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0 and df[col4][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop([col2, col3, col4], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'tu_quan_ao'})            
        if self.view:
            col1 = 'Nhìn ra thành phố'
            col2 = 'Nhìn ra hồ bơi'
            col3 = 'Tầm nhìn ra khung cảnh'
            col4 = 'Nhìn ra sông'
            col5 = 'Nhìn ra địa danh nổi tiếng'
            col6 = 'Nhìn ra vườn'
            col7 = 'Hướng nhìn sân trong'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0 and df[col4][i] == 0 and df[col5][i] == 0 and df[col6][i] == 0 and df[col7][i] == 0:
                    pass
                else:
                    if df[col2][i] == 1:
                        df[col1][i] = 1
                    if df[col1][i] == 1 or df[col3][i] == 1 or df[col7][i] == 1:
                        df[col1][i] = 2
                    if df[col6][i] == 1:
                        df[col1][i] = 3
                    if df[col4][i] == 1 or df[col5][i] == 1:
                        df[col1][i] = 4

            df.drop([col2, col3, col4, col5, col6, col7], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'view'})
        if self.ban_an:
            col1 = 'Bàn ăn'
            col2 = 'Khu vực phòng ăn'
            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop(col2, axis=1, inplace=True)
            df = df.rename(columns={col1 : 'ban_an'})
        if self.ui_quan_ao:
            col1 = 'Tiện nghi ủi'
            col2 = 'Bàn ủi'
            col3 = 'Bàn ủi li quần'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop([col2, col3], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'ui_quan_ao'})
        if self.giuong:
            col1 = 'Giường cực dài (> 2 mét)'
            col2 = 'Giường sofa'
            col3 = 'Giường xếp'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0:
                    df[col1][i] = 1
                else:
                    if df[col3][i] == 1:
                        df[col1][i] = 1
                    if df[col1][i] == 1 or df[col2][i] == 1:
                        df[col1][i] = 2

            df.drop([col2, col3], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'giuong'})
        if self.tien_ich_cho_tre_em:
            col1 = 'Ghế cao dành cho trẻ em'
            col2 = 'Cửa an toàn cho trẻ nhỏ'
            col3 = 'Nắp che ổ cắm điện an toàn'
            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop([col2, col3], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'tien_ich_cho_tre_em'})
        if self.ho_boi:
            col1 = 'Hồ bơi trên sân thượng'
            col2 = 'Hồ bơi có tầm nhìn'
            col3 = 'Hồ bơi riêng'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0 and df[col3][i] == 0:
                    pass
                else:
                    if df[col3][i] == 1:
                        df[col1][i] = 1
                    if df[col1][i] == 1 or df[col2][i] == 1:
                        df[col1][i] = 2

            df.drop([col2, col3], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'ho_boi'})            
        if self.khu_ngoai_troi:
            col1 = 'Khu vực ăn uống ngoài trời'
            col2 = 'Bàn ghế ngoài trời'
            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0:
                    pass
                else:
                    df[col1][i] = 1

            df.drop(col2, axis=1, inplace=True)
            df = df.rename(columns={col1 : 'khu_ngoai_troi'})
        if self.tien_ich_phong:
            col1 = 'Áo choàng tắm'
            col2 = 'Khăn tắm'
            col3 = 'Khăn tắm/Bộ khăn trải giường (có thu phí)'

            for i in df.index:
                if df[col1][i] == 0 and df[col2][i] == 0:
                    pass
                if df[col3][i] == 1:
                    df[col1][i] = 0
                if df[col1][i] == 1 or df[col2][i] == 1:
                    df[col1][i] = 1

            df.drop([col2, col3], axis=1, inplace=True)
            df = df.rename(columns={col1 : 'tien_ich_phong'})
        return df

In [63]:
nume_cols = ['Diện tích']
unorder_cate_cols = ['City']
order_cate_cols = ['Đồ vệ sinh cá nhân miễn phí', 'Vòi sen',
       'tien_ich_phong', 'ket_sat', 'toilet', 'Bàn làm việc',
       'Khu vực tiếp khách', 'TV', 'Dép', 'Tủ lạnh', 'Điện thoại',
       'Máy sấy tóc', 'lot_san', 'Ấm đun nước điện', 'Truyền hình cáp',
       'tu_quan_ao', 'view', 'Bồn tắm', 'Phòng tắm riêng',
       'Hệ thống cách âm', 'Minibar', 'WiFi miễn phí', 'Ghế sofa', 'bep_an',
       'Máy giặt', 'Bồn tắm hoặc Vòi sen', 'Đồng hồ báo thức', 'ban_an',
       'Bồn tắm spa', 'Ổ điện gần giường', 'Giấy vệ sinh',
       'Phòng tắm riêng trong phòng', 'ui_quan_ao', 'giuong',
       'Truyền hình vệ tinh', 'Ban công', 'Quyền sử dụng Executive Lounge',
       'Ra trải giường', 'may_pha_tra_cf', 'Máy sấy quần áo', 'Phòng tắm phụ',
       'Có phòng thông nhau qua cửa nối', 'Chậu rửa vệ sinh (bidet)',
       'Các tầng trên đi lên bằng thang máy',
       'Xe lăn có thể đi đến mọi nơi trong toàn bộ khuôn viên', 'Máy vi tính',
       'dieu_hoa', 'tien_ich_cho_tre_em', 'Hoàn toàn nằm ở tầng trệt',
       'Các tầng trên chỉ lên được bằng cầu thang', 'Nước rửa tay', 'ho_boi',
       'Sách', 'khu_ngoai_troi', 'Phòng xông hơi', 'Sân trong']
# 
nume_col_transformer = SimpleImputer(strategy = 'mean')
unorder_cate_col_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('one_hot_encoder', OneHotEncoder())])
order_cate_col_transformer = SimpleImputer(strategy = 'most_frequent')
col_transformer = ColumnTransformer(transformers = [
    ('nume_col_transformer', nume_col_transformer, nume_cols),
    ('unorder_cate_col_transformer', unorder_cate_col_transformer, unorder_cate_cols),
    ('order_cate_col_transformer', order_cate_col_transformer, order_cate_cols)])

# 
preprocess_pipeline = Pipeline(steps = [('col_adderdropper', ColAdderDropper()),
                                           ('col_transformer', col_transformer),
                                           ('col_normalizer', StandardScaler())])

In [64]:
preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)

In [61]:
preprocess_pipeline

Pipeline(steps=[('col_adderdropper', ColAdderDropper()),
                ('col_transformer',
                 ColumnTransformer(transformers=[('nume_col_transformer',
                                                  SimpleImputer(),
                                                  ['Diện tích']),
                                                 ('unorder_cate_col_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder())]),
                                                  ['City']),
                                                 ('order_cate_col_transformer',
                                                  SimpleImpute...
                                                   'Khu vực tiếp khách',
                                                   'TV', 'Dép', 'Tủ lạnh',
                                                   'Điện thoại',
                                                   'Máy sấy tóc', 'lot_san',
                                                   'Ấm đun nước điện',
                                                   'Truyền hình cáp',
                                                   'tu_quan_ao', 'view',
                                                   'Bồn tắm',
                                                   'Phòng tắm riêng',
                                                   'Hệ thống cách âm',
                                                   'Minibar', 'WiFi miễn phí',
                                                   'Ghế sofa', 'bep_an',
                                                   'Máy giặt',
                                                   'Bồn tắm hoặc Vòi sen',
                                                   'Đồng hồ báo thức', 'ban_an',
                                                   'Bồn tắm spa',
                                                   'Ổ điện gần giường', ...])])),
                ('col_normalizer', StandardScaler())])

---

## Tiền xử lý (tập validation)

In [ ]:
preprocessed_val_X = preprocess_pipeline.transform(val_X_df)

---

## Tiền xử lý + mô hình hóa

### Tìm mô hình tốt nhất

### Đánh giá mô hình tìm được